In [1]:
#Take the file and extract the journal dates and upload to Azure Cog Search index
# https://azsdemos.blob.core.windows.net/covid-19/metadata.csv

In [2]:
import globals

import csv
from dateutil import parser

import json
import requests
from pprint import pprint

from azure.storage.blob import BlockBlobService

metadata_file = "json/metadata.csv"

In [3]:
block_blob_service = BlockBlobService(account_name=globals.blob_account_name, account_key=globals.blob_account_key)

In [4]:
block_blob_service.get_blob_to_path(globals.blob_container_name, metadata_file, '/tmp/metadata.csv')

In [5]:
documents = {"value": []}
doc_counter = 0
batch_size = 1000

row_counter = 0
with open('/tmp/metadata.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        split_sha = row['sha'].split(';')
        
        # Sometimes the metadata can have two doc's in a single row
        for sha in split_sha:
            docID = ''
            pubDate = ''
            journal = ''
            fullTextURL = ''
            searchJSON = {"@search.action": "merge"}
            if len(sha.strip()) > 1:
                doc_counter += 1

                try:
                    docID = sha.strip()
                    searchJSON["docID"] = docID

                    pubDate = row['publish_time']
                    
                    if len(pubDate) > 11:
                        pubDate = parser.parse(pubDate[:8]).strftime('%Y-%m-%d') + 'T12:00:00Z'
                    elif len(pubDate) == 4:
                        pubDate = parser.parse(pubDate[:4]+"-01-01").strftime('%Y-%m-%d') + 'T12:00:00Z'
                    else:
                        pubDate = parser.parse(pubDate).strftime('%Y-%m-%d') + 'T12:00:00Z'
                    searchJSON["pubDate"] = pubDate
                    
                    searchJSON["journal"] = row['journal']
                    searchJSON["fullTextURL"] = row['url']
                    
                    documents["value"].append(searchJSON)
                    

                except Exception as ex:
                    print ("Error: ", ex)
                    
                if doc_counter % batch_size == 0:
                    print ("Applying docs:", doc_counter)
                    url = globals.endpoint + "indexes/" + globals.indexName + "/docs/index" + globals.api_version
                    response  = requests.post(url, headers=globals.headers, json=documents)
                    documents = {"value": []}
    
    

if doc_counter > 0:
    print ("Applying docs:", doc_counter)
    url = globals.endpoint + "indexes/" + globals.indexName + "/docs/index" + globals.api_version
    response  = requests.post(url, headers=globals.headers, json=documents)


Applying docs: 1000
Applying docs: 2000
Applying docs: 3000
Applying docs: 4000
Applying docs: 5000
Applying docs: 6000
Applying docs: 7000
Applying docs: 8000
Applying docs: 9000
Applying docs: 10000
Applying docs: 11000
Applying docs: 12000
Applying docs: 13000
Applying docs: 14000
Applying docs: 15000
Applying docs: 16000
Applying docs: 17000
Applying docs: 18000
Applying docs: 19000
Applying docs: 20000
Applying docs: 21000
Applying docs: 22000
Applying docs: 23000
Applying docs: 24000
Applying docs: 25000
Applying docs: 26000
Applying docs: 27000
Applying docs: 28000
Applying docs: 29000
Applying docs: 30000
Applying docs: 31000
Applying docs: 32000
Applying docs: 33000
Applying docs: 34000
Applying docs: 35000
Applying docs: 36000
Applying docs: 37000
Applying docs: 38000
Applying docs: 39000
Applying docs: 40000
Applying docs: 41000
Applying docs: 42000
Applying docs: 43000
Applying docs: 44000
Applying docs: 45000
Applying docs: 46000
Applying docs: 47000
Applying docs: 48000
A